In [1]:
%matplotlib inline
import pandas as pd
import csv as csv
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
#from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pylab as pl
import xgboost as xgb

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
#Loading data, the files are in a folder called 'csv'
ds    = pd.read_csv('../csv/train.csv', header=0, dtype={'StateHoliday':'str'})
store = pd.read_csv('../csv/store.csv', header=0)
test  = pd.read_csv('../csv/test.csv', header=0, dtype={'StateHoliday':'str'})

In [3]:
#Not necessary, but I like it better this way
ds.columns = [x.lower() for x in ds.columns]
store.columns = [x.lower() for x in store.columns]
test.columns  = [x.lower() for x in test.columns]

In [4]:
#string --> datetime
ds.date = pd.to_datetime(ds.date)
test.date = pd.to_datetime(test.date)

In [5]:
#Excluding the first day as discussed in the the Exploratory Analysis
ds = ds[ds.date>ds.date.min()]

In [6]:
#Dropping some columns as from the assignment. I will still keep some that the assignemnt asks to drop
#as they might turn out useful for the moment. I might drop them later
ds = ds.drop(['stateholiday','schoolholiday'],axis=1)
store = store.drop([
        'assortment','competitionopensincemonth','competitionopensinceyear',\
        'promo2','promo2sinceweek','promo2sinceyear','promointerval'
    ],axis=1)

In [7]:
test.drop(['stateholiday','schoolholiday'],axis=1, inplace=True)

In [8]:
#Merging datasets
ds = pd.merge(ds,store,on='store')
test = pd.merge(test,store,on='store')

In [9]:
#Checking for NaN's in the test file
len(test[test.open!=test.open])

11

In [10]:
#Populating the NaN's setting them open
test.loc[test.open!=test.open,'open']=1

In [11]:
#There are some NaN's in the competition distance as shown in the exploratory file
ds.fillna(0,inplace=True)
test.fillna(0,inplace=True)

Description and analysis is done in Exploratory Analisys file. I will concentrate on modelling in this file.

New features will be year, month, day of the month, easily extracted from date. I will not considered lagged values now as they are very time consuming to compute and might lead to accumulating error when I have to compute them day by day in the test for 6 weeks.

I will also count the number of days since the survey started and will attach to each store the average number of customers, the average nuber of customers per month and the average number of customers per year

I will semplify the situation by assuming that all stores are closed on Sunday, as argued in the Exploratory Analysis file this affect very few stores (~.1% of the whole set). I will then exclude Sundays from the analysis for the time being. NOTE: of course this will affect predictions for the stores open on Sundays. The best prediction for Sunday though is sales=0, as most of the stores are closed.

In [12]:
min_date = ds.date.min()

In [13]:
ds['year'] = ds.date.apply(lambda x:float(x.year))
ds['month'] = ds.date.apply(lambda x: float(x.month))
ds['dayofmonth'] = ds.date.apply(lambda x: float(x.day))

test['year'] = test.date.apply(lambda x:float(x.year))
test['month'] = test.date.apply(lambda x: float(x.month))
test['dayofmonth'] = test.date.apply(lambda x: float(x.day))

In [14]:
#Defining new df's so that I can see later the sundays for which I have to predict the value
ds_NS = ds[ds.dayofweek!=7]
test_NS = test[test.dayofweek!=7]

In [15]:
def days_mod_7(x,y):
    #Computes the days between two dates counting weeks as as 6 days. Takes two timestamps.
    days = (x-y).days
    return days - np.floor(days/7)

In [ ]:
#I konw this will give me a warning, but it does do what it is supposed to do
ds_NS['ndays'] = ds_NS.date.apply(lambda x: days_mod_7(x,min_date))
test_NS['ndays'] = test_NS.date.apply(lambda x: days_mod_7(x,min_date))

In [17]:
#Average customer values per store, month and year, for days with customers
cust = ds_NS.groupby(['store']).agg({
        'customers':{
            'cust_store':lambda y: np.mean([x for x in y if x!=0])
        }
    })
cust.columns = cust.columns.get_level_values(1)
cust.reset_index(inplace=True)

cust_month = ds_NS.groupby(['store','month']).agg({
        'customers':{
            'cust_month':lambda y: np.mean([x for x in y if x!=0])
        }
    })
cust_month.columns = cust_month.columns.get_level_values(1)
cust_month.reset_index(inplace=True)

cust_year = ds_NS.groupby(['store','year']).agg({
        'customers':{
            'cust_year':lambda y: np.mean([x for x in y if x!=0])
        }
    })
cust_year.columns = cust_year.columns.get_level_values(1)
cust_year.reset_index(inplace=True)

In [18]:
#Merging to set the new features
ds_NS = pd.merge(ds_NS,cust,on=['store'])
ds_NS = pd.merge(ds_NS,cust_month,on=['store','month'])
ds_NS = pd.merge(ds_NS,cust_year,on=['store','year'])

test_NS = pd.merge(test_NS,cust,on=['store'])
test_NS = pd.merge(test_NS,cust_month,on=['store','month'])
test_NS = pd.merge(test_NS,cust_year,on=['store','year'])

In [19]:
#Changing to numerical values
storetype_dict = {'a':0.0,'b':1.0,'c':2.0,'d':3.0}
ds_NS = ds_NS.replace({'storetype':storetype_dict})
test_NS = test_NS.replace({'storetype':storetype_dict})

In [20]:
#Dropping columns I do not need anymore
ds_NS = ds_NS.drop(['open','date','customers'],axis=1)
test_NS = test_NS.drop(['open','date'],axis=1)

In [21]:
#Defining the measure that the challenge uses to test, i.e. rmspe

def ToWeight(y):
    '''Auxilliary to computing rmspe, computes the weight'''
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(ytest, y):
    '''Computes random mean square percentage error'''
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w*(y-ytest)**2 ))
    return rmspe

def rmspe_xg(ytest, y):
    '''rmspe to use in xgboost'''
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    ytest = np.exp(ytest) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w*(y-ytest)**2))
    return "rmspe", rmspe

I will prepare here the set to train the model. I will use xgboost. Whereas I will predict "by hand" that on sundays the sales will be 0.

In [22]:
test_NS.sort_values('id').head()

,id,store,dayofweek,promo,storetype,competitiondistance,year,month,dayofmonth,ndays,cust_store,cust_month,cust_year
0,1,1,4,1,2,1270,2015,9,17,847,564.049936,510.980392,531.171429
41,2,3,4,1,0,14130,2015,9,17,847,750.077022,725.568627,722.668571
82,3,7,4,1,0,24000,2015,9,17,847,948.561069,951.823529,931.079545
123,4,8,4,1,0,7520,2015,9,17,847,658.197704,642.039216,719.596591
164,5,9,4,1,0,2030,2015,9,17,847,579.816431,563.294118,643.182857


In [23]:
ds_NS.head()

,store,dayofweek,sales,promo,storetype,competitiondistance,year,month,dayofmonth,ndays,cust_store,cust_month,cust_year
0,1,5,5263,1,2,1270,2015,7,31,806,564.049936,528.061728,531.171429
1,1,4,5020,1,2,1270,2015,7,30,805,564.049936,528.061728,531.171429
2,1,3,4782,1,2,1270,2015,7,29,804,564.049936,528.061728,531.171429
3,1,2,5011,1,2,1270,2015,7,28,804,564.049936,528.061728,531.171429
4,1,1,6102,1,2,1270,2015,7,27,803,564.049936,528.061728,531.171429


In [24]:
features = ds_NS.drop('sales',axis=1).columns

In [26]:
features

Index([u'store', u'dayofweek', u'promo', u'storetype', u'competitiondistance',
       u'year', u'month', u'dayofmonth', u'ndays', u'cust_store',
       u'cust_month', u'cust_year'],
      dtype='object')

I will use boosted trees to model and make predictions. The kaggle challenge asks to minimize the root mean square percentage error (RMSPE) instead of RMSE: in order to use this I will predict the log of the sales and then take the exponential of my results.

In [28]:
'''Parameters for xgboost. Ideally one can use more trees (I had an example with 2000) and possibly rescale eta for
more precise results, leaving these parameters here for now. Notice that the training might still take several minutes
with these parameters'''

params = {"objective": "reg:linear",
          "eta": .05,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1
          }
num_trees = 200

In [29]:
from sklearn import cross_validation

'''Setting a training set and crossvalidation. Notice that I used only the stores that are in the test to train my
model for simplicity. This can clearly be easily modified.'''

X_train, X_test = cross_validation.train_test_split(ds_NS[ds_NS.store.isin(test_NS.store.unique())], test_size=0.03)

In [30]:
dtrain = xgb.DMatrix(X_train[features], np.log(X_train["sales"] + 1))
dvalid = xgb.DMatrix(X_test[features], np.log(X_test["sales"] + 1))
dtest = xgb.DMatrix(test_NS[features])
watchlist = [(dvalid, 'eval'), (dtrain, 'train')]

In [31]:
#Training the model
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, verbose_eval=True, feval=rmspe_xg)

Will train until train error hasn't decreased in 50 rounds.
[0]	eval-rmspe:0.982772	train-rmspe:0.981925
[1]	eval-rmspe:0.982579	train-rmspe:0.981733
[2]	eval-rmspe:0.982311	train-rmspe:0.981467
[3]	eval-rmspe:0.981944	train-rmspe:0.981102
[4]	eval-rmspe:0.981460	train-rmspe:0.980620
[5]	eval-rmspe:0.980825	train-rmspe:0.979990
[6]	eval-rmspe:0.980008	train-rmspe:0.979178
[7]	eval-rmspe:0.978963	train-rmspe:0.978139
[8]	eval-rmspe:0.977653	train-rmspe:0.976837
[9]	eval-rmspe:0.976064	train-rmspe:0.975259
[10]	eval-rmspe:0.974117	train-rmspe:0.973323
[11]	eval-rmspe:0.971738	train-rmspe:0.970959
[12]	eval-rmspe:0.968961	train-rmspe:0.968195
[13]	eval-rmspe:0.965647	train-rmspe:0.964902
[14]	eval-rmspe:0.961783	train-rmspe:0.961061
[15]	eval-rmspe:0.957305	train-rmspe:0.956607
[16]	eval-rmspe:0.952175	train-rmspe:0.951509
[17]	eval-rmspe:0.946510	train-rmspe:0.945875
[18]	eval-rmspe:0.940157	train-rmspe:0.939554
[19]	eval-rmspe:0.933112	train-rmspe:0.932547
[20]	eval-rmspe:0.925135	train

In [32]:
print "Validating"
train_probs = gbm.predict(xgb.DMatrix(X_test[features]))
indices = train_probs < 0
train_probs[indices] = 0
error = rmspe(np.exp(train_probs) - 1, X_test['sales'].values)
print error

Validating
0.159083291506


In [33]:
print("Make predictions on the test set")
real_test = gbm.predict(xgb.DMatrix(test_NS[features]))

Make predictions on the test set


In [34]:
len(test_NS)

35096

In [35]:
len(real_test)

35096

In [36]:
#Taking expinentials and adding a column to test
test_NS['Sales'] = np.exp(real_test)-1

In [37]:
#Setting the whole test df to zero. I will select only the Sundays from here.
test['Sales'] = 0.0

In [38]:
#Generating solution my adding Sundays
sol = pd.concat([
        test.loc[test.dayofweek==7,['id','Sales']],
        test_NS[['id','Sales']]
    ])

In [39]:
sol.head()

,id,Sales
4,3425,0
11,9417,0
18,15409,0
25,21401,0
32,27393,0


In [40]:
len(sol)

41088

In [41]:
sol.id.nunique()

41088

In [42]:
duplicates = sol.set_index('id').index.get_duplicates()

In [43]:
len(duplicates)

0

In [44]:
sol.columns = ['Id','Sales']

In [45]:
sol = sol.sort_values('Id').reset_index().drop('index',axis=1)

In [46]:
sol.head()

,Id,Sales
0,1,4340.654785
1,2,7953.810547
2,3,9951.772461
3,4,7370.900391
4,5,6149.442871


In [47]:
#Genereting the file to submit to kaggle
sol.to_csv('solXGB200.csv',index=False)